In [9]:
import pandas as pd
from refml.topic_modeling.preprocess import stem_and_tokenize
from refml.topic_modeling.model import refml_nmf_tsne
import getpass
import cx_Oracle

In [10]:
def hero_to_df(Userid, Password, Host, Port, SID, query):

    dsn_tns = cx_Oracle.makedsn(host=Host, port=Port, sid=sid_name)
    conn = cx_Oracle.connect(user=Userid, password=Password, dsn=dsn_tns)

    df = pd.read_sql_query(query, conn)
    df.ABSTRACT = df.ABSTRACT.astype(str)
    df.TITLE = df.TITLE.astype(str)
    
    conn.close()
    
    return(df)

## HEROPRD
Userid = "rrice"
Password = getpass.getpass()
Host = "herodb.rtpnc.epa.gov"
Port = "1521"
sid_name = "heroprd"

def defaultquery(query):
    return hero_to_df(Userid, Password, Host, Port, sid_name, query)

o3litquery = '''select reference_id, year, title, abstract
    from tbl_reference 
    where sdelete = 'No' 
    and reference_id in
    (select reference_id from tbl_reference_usage where usage_id = 19134 and isdeleted = 0)'''

o3includedquery = '''select reference_id, year, title, abstract
    from tbl_reference 
    where sdelete = 'No' 
    and reference_id in
    (select reference_id from tbl_reference_usage where usage_id = 24487 and isdeleted = 0)'''

········


In [14]:
# query HERO
df = defaultquery(o3litquery) #dataframe of all references included in lit search
df2 = defaultquery(o3includedquery) #dataframe of all references included in final document

In [15]:
# add column to indicate references that were included in the final draft of the ISA
df['included'] = df.REFERENCE_ID.isin(df2.REFERENCE_ID)
df.head()

,REFERENCE_ID,YEAR,TITLE,ABSTRACT,included
0,4310270,2011.0,Inbreeding depression in Solanum carolinense (...,The clonal weed Solanum carolinense exhibits p...,False
1,4310275,2013.0,EFFECTS OF MIXED SALINE AND ALKALINE STRESS ON...,"The effects of hyperalkaline, thermo-mineral w...",False
2,4310369,2013.0,Metabolomics as a tool to investigate abiotic ...,Metabolites reflect the integration of gene ex...,False
3,4310370,2016.0,Manipulation of ripening via antitranspirants ...,Background and Aims: Accelerated berry sugar a...,False
4,4310409,2015.0,Genotype x environment interactions for wheat ...,The phenotypic performance of a genotype is no...,False


In [16]:
# text preprocessing - stem, tokenize, and remove stop words
##currently set up to combine title and abstract into one field for preprocessing
df['Text'] = df['TITLE'] + " " + df['ABSTRACT'].astype(str)
df['Text_Processed'] = df['Text'].apply(stem_and_tokenize)
df.Text_Processed.head()

0    inbreed depress solanum carolinens solanacea f...
1    effect OF mix salin alkalin stress ON morpholo...
2    metabolom tool investig abiot stress toler pla...
3    manipul ripen via antitranspir cv barbera viti...
4    genotyp x environ interact wheat grain yield a...
Name: Text_Processed, dtype: object

In [24]:
# run nmf model and use TSNE dimensionality reduction to visualize
reduced = refml_nmf_tsne(df.Text_Processed, n_topics = 20, n_top_words = 10, n_dimensions = 2, perplexity = 30)

In [25]:
export = reduced.join(df)
export.to_pickle('OzoneISA_topic_clustering_export.pkl')

In [21]:
df2

,REFERENCE_ID,YEAR,TITLE,ABSTRACT
0,1061833,2012,Regional and global modeling estimates of poli...,"Policy Relevant Background (PRB) ozone, as def..."
1,2534734,2014,High winter ozone pollution from carbonyl phot...,The United States is now experiencing the most...
2,4254337,2017,"Contributions of foreign, domestic and natural...",The Goddard Earth Observing System global chem...
3,1064116,2012,Ozone production from wildfires: A critical re...,Tropospheric ozone (O3) negatively impacts hum...
4,4255333,2017,Stratospheric intrusion-influenced ozone air q...,Stratospheric intrusions have been the interes...
...,...,...,...,...
1699,2447683,2014,Integrated review plan for the primary nationa...,The U.S. Environmental Protection Agency (EPA)...
1700,5408574,2014,"Letter from Dr. H. Christopher Frey, Chair, Cl...",None
1701,42135,1987,Selection of the best exposure-response model ...,None
1702,6309703,2016,The role of ghrelin in the regulation of gluco...,Ghrelin is a 28-amino acid (aa) stomach-derive...
